In [6]:
import pandas as pd
import numpy as np

from sklearn import model_selection

# AutoML 

### auto-sklearn

In [12]:
titanic_df = pd.read_csv('titanic.csv')

In [8]:
import autosklearn.classification

In [2]:
automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=60)

In [9]:
%%time
automl.fit(X_train[ ['Pclass', 'SibSp','Fare' ] ], y_train)

NameError: name 'X_train' is not defined

In [ ]:
from sklearn import metrics

In [ ]:
y_hat = automl.predict(X_test[ ['Pclass', 'SibSp','Fare' ] ])
print("Accuracy score", metrics.accuracy_score(y_test, y_hat))

In [ ]:
automl.show_models()

### TPOT

In [2]:
titanic_df = pd.read_csv('titanic.csv')

In [3]:
titanic_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    titanic_df.drop(['Survived','Name','PassengerId','Sex','Ticket','Cabin','Embarked'], axis=1),
    titanic_df['Survived'])

In [5]:
from tpot import TPOTClassifier

In [15]:
X_train

,Pclass,Age,SibSp,Parch,Fare
168,1,NaN,0,0,25.9250
277,2,NaN,0,0,0.0000
25,3,38.00,1,5,31.3875
328,3,31.00,1,1,20.5250
505,1,18.00,1,0,108.9000
...,...,...,...,...,...
644,3,0.75,2,1,19.2583
145,2,19.00,1,1,36.7500
628,3,26.00,0,0,7.8958
13,3,39.00,1,5,31.2750


In [12]:
pipeline_optimizer = TPOTClassifier()


tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

Imputing missing values in feature set




TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: LinearSVC(input_matrix, C=0.1, dual=True, loss=squared_hinge, penalty=l2, tol=0.0001)
Imputing missing values in feature set
0.6322869955156951


In [44]:
pd.concat( [X_train, y_train] , axis =1).to_csv('dataset_for_tpot.csv')

In [45]:
tpot.export('my_tpot_code.py', data_file_path='dataset_for_tpot.csv')

In [50]:
# %load my_tpot_code.py
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = pd.read_csv('dataset_for_tpot.csv', dtype=np.float64)
features = tpot_data.drop('Survived', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['Survived'], random_state=None)

imputer = SimpleImputer(strategy="median")
imputer.fit(training_features)
training_features = imputer.transform(training_features)
testing_features = imputer.transform(testing_features)

# Average CV score on the training set was: 0.7485242958141622
exported_pipeline = ExtraTreesClassifier(bootstrap=True, criterion="entropy", max_features=0.55, min_samples_leaf=1, min_samples_split=15, n_estimators=100)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)


In [51]:
results

array([0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
       0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
       0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1.,
       0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1.,
       1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.])